# Promises

## What is a Promise?
Originally, JavaScript used callback functions to handle asynchronous actions. The problem with callbacks is that they encourage complexly nested code which quickly becomes difficult to read, debug, and scale. With ES6, JavaScript integrated native promises which allow us to write significantly more readable code. Promises are objects that represent the eventual outcome of an asynchronous operation. A Promise object can be in one of three states:

- **Pending:** The initial state— the operation has not completed yet.
- **Fulfilled:** The operation has completed successfully and the promise now has a resolved value. For example, a request’s promise might resolve with a JSON object as its value.
- **Rejected:** The operation has failed and the promise has a reason for the failure. This reason is usually an Error of some kind.

## Constructing a Promise Object
 To create a new Promise object, we use the new keyword and the Promise constructor method. The Promise constructor method takes a function parameter called the executor function which runs automatically when the constructor is called. The executor function generally starts an asynchronous operation and dictates how the promise should be settled.

The executor function has two function parameters, usually referred to as the resolve() and reject() functions. The resolve() and reject() functions aren’t defined by the programmer. When the Promise constructor runs, JavaScript will pass its own resolve() and reject() functions into the executor function.

- `resolve()` is a function with one argument. Under the hood, if invoked, resolve() will change the promise’s status from pending to fulfilled, and the promise’s resolved value will be set to the argument passed into resolve().
- `reject()` is a function that takes a reason or error as an argument. Under the hood, if invoked, reject() will change the promise’s status from pending to rejected, and the promise’s rejection reason will be set to the argument passed into reject().

In [ ]:
var {inventory} = require('./inventory.js');
  
// Create executor function
function myExecutor(resolve, reject) {
  if (inventory.sunglasses.stock > 0) {
    resolve('Sunglasses are in stock! (⌐■_■)')
  } else {
    reject('Sunglasses are sold out')
  }
}

// Create function which returns promise
function orderSunglasses() {
  return new Promise(myExecutor)
}

// Store resolved promise object
var orderPromise = orderSunglasses()

// log promise
console.log(orderPromise)

## The setTimeout() Function
`setTimeout()` is a Node API (a comparable API is provided by web browsers) that uses callback functions to schedule tasks to be performed after a delay. It's useful to simulate asynchronous tasks. setTimeout() has two parameters: a callback function and a delay in milliseconds. In the example below, though setTimeout is called first, it's output is delivered after some delay.

In [ ]:
console.log("This is the first line of code");

function usingSTO() {
  console.log("TuRTle InComIng... 🐢")
}

setTimeout(usingSTO, 2000)

console.log("This is the last line of code... but is it last to execute?? ...🤔")

# Consuming Promises
The initial state of an asynchronous promise is pending, but we have a guarantee that it will settle. How do we tell the computer what should happen then? Promise objects come with an aptly named `.then()` method. It allows us to say, “I have a promise, when it settles, then here’s what I want to happen…” `.then()` is a higher-order function— it takes two callback functions as arguments. We refer to these callbacks as handlers. When the promise settles, the appropriate handler will be invoked with that settled value.

- The first handler, sometimes called onFulfilled, is a success handler, and it should contain the logic for the promise *resolving*.
- The second handler, sometimes called onRejected, is a failure handler, and it should contain the logic for the promise *rejecting*.

## Successful Promises
To handle a “successful” promise, or a promise that resolved, we invoke .then() on the promise, passing in a success handler callback function.

In [ ]:
var prom = new Promise((resolve, reject) => {
    resolve('Yay! 🥳');
  });
  
var handleSuccess = (resolvedValue) => {
  console.log(resolvedValue);
};

prom.then(handleSuccess); // Prints: 'Yay!'

## Rejected Promises
When promises fail, a handler can also be passed. The code below simulates a promise that *sometimes* is broken. 😲 

Notice that .then() is passed with a second callback function for how to deal with a failed promise.

In [ ]:
var iffyProm = new Promise((resolve, reject) => {
    let num = Math.random();
    if (num < .5 ){
      resolve('Yay! 🙌');
    } else {
      reject('Ohhh noooo! (っ °Д °;)っ');
    }
  });
  
var dealWithSuccess = (resolvedValue) => {
  console.log(resolvedValue);
};

var dealWithFailure = (rejectionReason) => {
  console.log(rejectionReason);
};
  
iffyProm.then(dealWithSuccess, dealWithFailure);

/*
An alternative syntax for the above statement is shown below.

iffyProm.then(dealWithSuccess).then(null, dealWithFailure);
*/

## Using .catch() with Promises

One way to write cleaner code is to follow a principle called separation of concerns. As such, with separation of concerns, we should organize code into distinct sections each handling a specific task. In the case of handling promises, we should seperate logic for dealing with successful or failed promise resolutions. To do so, we can use .catch as follows:

In [ ]:
var {checkInventory} = require('./inventory.js');

var order = {
  items: [['sunglasses', 1], ['bags', 2]] // adjust order quantities to see the failure case
}; 

var handleSuccess = (resolvedValue) => {
  var items = resolvedValue[0].items
  console.log('shopping cart 🛒:')
  for (let i = 0; i < items.length; i++) {
    console.log(`- ${items[i][0]}: ${items[i][1]}`)
  }
};

var handleFailure = (rejectReason) => {
  console.log(rejectReason);
};

// Write your code below:
checkInventory(order)
  .then(handleSuccess)
  .catch(handleFailure)

## Chaining Promises
This process of chaining promises together is called composition; promises are designed with composition in mind! In the example below, we import several modules that handle processing steps for an order; namely checking inventory, processing the payment, and resolving shipment via a tracking number. 

With chaining, each step is executed sequentially, but this sequence is nonetheless non-blocking because of our use of promises. To model the real-world behavior of network traffic to remote databases, each function is called with a randomly generated timeout that models the varying execution speed of asynchronous functions.

In [ ]:
var {checkInventory} = require('./inventory.js');
var {processPayment} = require('./payment.js');
var {shipOrder} = require('./shipping.js');

var order = {
  items: [['sunglasses', 1], ['bags', 2]],
  giftcardBalance: 79.82
};

checkInventory(order)
.then((resolvedValueArray) => {
  // Write the correct return statement here:
  return processPayment(resolvedValueArray)
})
.then((resolvedValueArray) => {
  // Write the correct return statement here:
  return shipOrder(resolvedValueArray)
  
})
.then((successMessage) => {
  console.log(successMessage);
})
.catch((errorMessage) => {
  console.log(errorMessage);
});

## Using Promise.all()
What if we’re dealing with multiple promises, but we don’t care about the order? To maximize efficiency we should use concurrency, multiple asynchronous operations happening together. With promises, we can do this with the function Promise.all().

`Promise.all()` accepts an array of promises as its argument and returns a single promise. That single promise will settle in one of two ways:

- If every promise in the argument array resolves, the single promise returned from Promise.all() will resolve with an array containing the resolve value from each promise in the argument array.
- If any promise from the argument array rejects, the single promise returned from Promise.all() will immediately reject with the reason that promise rejected. This behavior is sometimes referred to as failing fast.

In [ ]:
var {checkDistributorStock} = require('./distributor.js');

var onFulfill = (itemsArray) => {
  console.log(`Items checked: ${itemsArray. join(', ')}...`);
  console.log(`Every item was available from the distributor. Placing order now.`);
};

var onReject = (rejectionReason) => {
	console.log(rejectionReason);
};

// Write your code below:
var checkSunglasses = checkDistributorStock('sunglasses', 'Favorite Supply Co.')

var checkPants = checkDistributorStock('pants', 'Favorite Supply Co.')

var checkBags = checkDistributorStock('bags', 'Favorite Supply Co.')

Promise.all([checkSunglasses, checkPants, checkBags])
  .then(onFulfill)
  .catch(onReject)



# Async ... Await
JavaScript is continually improving, and ES8 provides a new syntax for handling our asynchronous action, async...await. The async...await syntax allows us to write asynchronous code that reads similarly to traditional synchronous, imperative programs.

## Async Syntactic Sugar
The async...await syntax is syntactic sugar — it doesn’t introduce new functionality into the language, but rather introduces a new syntax for using promises and generators. Both of these were already built in to the language. Despite this, async...await powerfully improves the readability and scalability of our code.

In [ ]:
var fs = require('fs');
var {promisifiedReadfile} = require('./utils.js');
      
// Here we use fs.readfile() and callback functions:
fs.readFile('./data/file1.txt', 'utf-8', (err, data) => {
  if (err) throw err;
  let firstSentence = data;
  fs.readFile('./data/file2.txt',  'utf-8', (err, data) => {
    if (err) throw err;
    let secondSentence = data;
    console.log(`Legacy callback syntax: \n\t- ${firstSentence}. ${secondSentence}.`);
  });
});

// Here we use native promises with our "promisified" version of readfile:
let firstSentence;
promisifiedReadfile('./data/file1.txt', 'utf-8')
  .then((data) => {
    firstSentence = data;
    return promisifiedReadfile('./data/file2.txt', 'utf-8');
  })
  .then((data) => {
    let secondSentence = data;
    console.log(`ES6 promisified syntax: \n\t- ${firstSentence}. ${secondSentence}.`)
  })
  .catch((err) => {console.log(err)});

// Here we use promisifiedReadfile() again but instead of using the native promise .then() syntax, we declare and invoke an async/await function:
async function readFiles() {
  let firstSentence = await promisifiedReadfile('./data/file1.txt', 'utf-8');
  let secondSentence = await promisifiedReadfile('./data/file2.txt', 'utf-8');
  console.log(`ES8 async syntactic sugar: \n\t- ${firstSentence}. ${secondSentence}.`);
}

readFiles();


## Using Async
The async keyword is used to write functions that handle asynchronous actions. We wrap our asynchronous logic inside a function prepended with the async keyword. Then, we invoke that function. async functions always return a promise. This means we can use traditional promise syntax, like .then() and .catch with our async functions. An async function will return in one of three ways:

- If there’s nothing returned from the function, it will return a promise with a resolved value of undefined.
- If there’s a non-promise value returned from the function, it will return a promise resolved to that value.
- If a promise is returned from the function, it will simply return that promise

In [ ]:
// promisified syntax
function withConstructor(num){
  return new Promise((resolve, reject) => {
    if (num === 0){
      resolve('zero');
    } else {
      resolve('not zero');
    }
  });
}

withConstructor(0)
  .then((resolveValue) => {
  console.log(` withConstructor(0) returned a promise which resolved to: ${resolveValue}.`);
});

// With async, a promise is automatically returned
async function withAsync(num) {
  if (num === 0) {
    return 'zero'
  } else {
    return 'not zero'
  }

}

withAsync(100)
  .then((resolveValue) => {
  console.log(` withAsync(100) returned a promise which resolved to: ${resolveValue}.`);
})


## Await Operator

The await keyword can only be used inside an async function. await is an operator: it returns the resolved value of a promise. Since promises resolve in an indeterminate amount of time, await halts, or pauses, the execution of our async function until a given promise is resolved.

In [1]:
var {brainstormDinner} = require('./dinner.js');

// Native promise version:
function nativePromiseDinner() {
  brainstormDinner().then((meal) => {
	  console.log(`I'm going to make ${meal} for dinner.`);
  });
}

// async/await version:
async function announceDinner() {
  // Write your code below:
  var meal = await brainstormDinner()
  console.log(`I'm going to make ${meal} for dinner! 🫘`);  
  
}

announceDinner()


I have to decide what's for dinner...


Promise { <pending> }

Should I make salad...?
Should I make ramen...?
Should I make eggs...?
Should I make chicken...?
I'm going to make beans for dinner.


## Handling Dependent Promises
The true beauty of async...await is when we have a series of asynchronous actions which depend on one another. For example, we may make a network request based on a query to a database. In that case, we would need to wait to make the network request until we had the results from the database. 

With native promise syntax, we use a chain of .then() functions making sure to correctly return each one. However, with 'Async ... Await' syntax, handling async dependencies is as simple as adding await and ordering each async process in the sequential manner intended. Await helps us to 'wait' for a promise resolution before the next async process ensues. 

In [ ]:
var {shopForBeans, soakTheBeans, cookTheBeans} = require('./dinner.js');

// declare async series of processes
async function makeBeans() {
  var type = await shopForBeans()
  var isSoft = await soakTheBeans(type)
  var dinner = await cookTheBeans(isSoft)

  console.log(dinner)

}

makeBeans()

## Handling Errors
When .catch() is used with a long promise chain, there is no indication of where in the chain the error was thrown. This can make debugging challenging.

With async...await, we use try...catch statements for error handling. By using this syntax, not only are we able to handle errors in the same way we do with synchronous code, but we can also catch both synchronous and asynchronous errors. This makes for easier debugging!

In [ ]:
var {cookBeanSouffle} = require('./dinner.js');

// Write your code below:
async function hostDinnerParty() {
  try {
    let beanSouffle = await cookBeanSouffle()
    console.log(`${beanSouffle} is served!`)
  } catch (error) {
    console.log(error)
    console.log('Ordering a pizza!')
  }
}

hostDinnerParty()